# My Learning Notes: BPE Tokenizer From Scratch (Simple)

**What I'm practicing**: Building a simple, readable BPE tokenizer to understand how subword vocabularies are learned.

**My goal**: See the merge process end-to-end, even if the implementation is slow.

**Reference**: Based on concepts from *Build a Large Language Model From Scratch* (Sebastian Raschka).

## Byte Pair Encoding (BPE) — Simple, Educational Version

I’m using this notebook as a **learning aid**:

- I want a clear, inspectable implementation (not a fast one).
- I’m focusing on the core BPE ideas: merges, vocabulary growth, encode/decode.
- I’ll keep examples small so I can reason about them easily.

**Note to self**: This is a naive implementation built for clarity. There is a more advanced version in this repo: [notebooks/ch02/05_bpe-from-scratch/bpe-from-scratch.ipynb](notebooks/ch02/05_bpe-from-scratch/bpe-from-scratch.ipynb).

# 1. Big idea behind BPE

I’m trying to map text into token IDs **shorter than character-level tokens**, while still being reversible.

**My mental model**: Start with byte-level tokens, then repeatedly merge the most frequent adjacent pairs to create subword tokens. This gives a compact vocabulary while keeping tokenization reversible.

## 1.1 Bits and bytes

Before BPE, I want to make sure I understand byte-level tokenization.

If I convert text to a `bytearray`, each byte becomes an integer between 0 and 255. This is the starting point for byte-level vocabularies.

In [ ]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


Conversion to list of integers: each byte becomes an integer.

In [ ]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


**My observation**: This simple byte-conversion works, but it's inefficient.
- It produces one token for every single character/byte.
- For "This is some text", I get 17 tokens.
- This would make the sequence length for LLMs very long, making training harder and context windows fill up faster.

In [ ]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


**Comparison with production tokenizers**:
Real tokenizers (like GPT-2's) compress common words into single tokens.
For the same text, GPT-2 uses only **4 tokens**: `[1212, 318, 617, 2420]`.

I can verify this using `tiktoken`:

**Vocabulary Initialization**:
BPE starts with all 256 possible byte values. This ensures any string can be tokenized (worst case, fall back to bytes).

I can verify the first few tokens of GPT-2's vocabulary correspond to ASCII characters:

- Above, note that entries 256 and 257 are not single-character values but double-character values (a whitespace + a letter), which is a little shortcoming of the original GPT-2 BPE Tokenizer (this has been improved in the GPT-4 tokenizer)

&nbsp;
## 1.2 Building the vocabulary

- The goal of the BPE tokenization algorithm is to build a vocabulary of commonly occurring subwords like `298: ent` (which can be found in *entangle, entertain, enter, entrance, entity, ...*, for example), or even complete words like 

```
318: is
617: some
1212: This
2420: text
```

## 1.3 Algorithm Outline

**My Summary of the BPE Process**:

1.  **Start small**: Vocabulary = all 256 bytes.
2.  **Count frequent pairs**: Look at the current token sequence, find the most frequent adjacent pair (e.g., `("e", "s")`).
3.  **Merge**: Create a new token for this pair (e.g., `257` = `"es"`).
4.  **Replace**: Replace all occurrences of the pair with the new token ID.
5.  **Repeat**: Keep merging until I reach the desired vocabulary size.

**Decoding** is simple: just map the token IDs back to their bytes/strings.

## 1.4 Walking Through an Example

Suppose I want to tokenize: `the cat in the hat`

**Iteration 1**:
- Most frequent pair: `t` and `h` (appears in "the" twice).
- Action: Merge `t` + `h` -> `th` (assign ID 256).
- Text becomes: `<th, 256>e cat in <th, 256>e hat`

**Iteration 2**:
- Most frequent pair: `256` ("th") and `e`.
- Action: Merge `256` + `e` -> `the` (assign ID 257).
- Text becomes: `<the, 257> cat in <the, 257> hat`

**Iteration 3**:
- Most frequent pair: `257` ("the") and ` ` (space).
- Action: Merge `257` + ` ` -> `the ` (assign ID 258).
- Text becomes: `<the , 258>cat in <the , 258>hat`

This process continues, building larger tokens from smaller ones.

# 2. Implementing BPE in Python

The class below implements this logic. It has two main modes:
1.  `train()`: Learns the merges from raw text.
2.  `encode()`: Apply learned merges to new text.

I've added comments to explain each step.

In [ ]:
from collections import Counter, deque
from functools import lru_cache


class BPETokenizerSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with 'Ġ'
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # Initialize vocab with unique characters, including 'Ġ' if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]

        # Extend unique_chars with characters from processed_text that are not already included
        unique_chars.extend(char for char in sorted(set(processed_text)) if char not in unique_chars)

        # Optionally, ensure 'Ġ' is included if it is relevant to your text processing
        if 'Ġ' not in unique_chars:
            unique_chars.append('Ġ')

        # Now create the vocab and inverse vocab dictionaries
        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:  # No more pairs to merge. Stopping training.
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def encode(self, text):
        """
        Encode the input text into a list of token IDs.

        Args:
            text (str): The text to encode.

        Returns:
            List[int]: The list of token IDs.
        """
        tokens = []
        # Split text into tokens, keeping newlines intact
        words = text.replace("\n", " \n ").split()  # Ensure '\n' is treated as a separate token

        for i, word in enumerate(words):
            if i > 0 and not word.startswith("\n"):
                tokens.append("Ġ" + word)  # Add 'Ġ' to words that follow a space or newline
            else:
                tokens.append(word)  # Handle first word or standalone '\n'

        token_ids = []
        for token in tokens:
            if token in self.inverse_vocab:
                # token is contained in the vocabulary as is
                token_id = self.inverse_vocab[token]
                token_ids.append(token_id)
            else:
                # Attempt to handle subword tokenization via BPE
                sub_token_ids = self.tokenize_with_bpe(token)
                token_ids.extend(sub_token_ids)

        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        can_merge = True
        while can_merge and len(token_ids) > 1:
            can_merge = False
            new_tokens = []
            i = 0
            while i < len(token_ids) - 1:
                pair = (token_ids[i], token_ids[i + 1])
                if pair in self.bpe_merges:
                    merged_token_id = self.bpe_merges[pair]
                    new_tokens.append(merged_token_id)
                    # Uncomment for educational purposes:
                    # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                    i += 2  # Skip the next token as it's merged
                    can_merge = True
                else:
                    new_tokens.append(token_ids[i])
                    i += 1
            if i < len(token_ids):
                new_tokens.append(token_ids[i])
            token_ids = new_tokens

        return token_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        decoded_string = ""
        for token_id in token_ids:
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token.startswith("Ġ"):
                # Replace 'Ġ' with a space
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced


# 3. Trying it Out

Let's test the implementation on a real file.

I'll download a short story ("The Verdict") to use as training data. This gives enough text to find meaningful patterns.

In [ ]:
import os
import urllib.request

if not os.path.exists("../01_main-chapter-code/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "../01_main-chapter-code/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("../01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

Now I'm training the tokenizer.

**Settings**:
- `vocab_size=1000`: This is small (GPT-2 is 50k), but good for a demo.
- Since we start with 256 byte tokens, the model will learn **744 new merges** (1000 - 256).

In [ ]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

I can check the size of the learned vocabulary:

In [ ]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


In [ ]:
print(len(tokenizer.bpe_merges))

742


Now I'll test the **encoding** on a new sentence.
I expect the number of tokens to be significantly less than the number of characters.

In [ ]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [ ]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 42
Number of token IDs: 20


**My result**: 42 characters were compressed into ~20 tokens. This is the power of BPE - roughly 2x compression for English text!

Now **decoding** back to text:

In [ ]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [ ]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.


I can look at exactly what each token represents:

In [ ]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .


**Observation**: Because my training data (one short story) and vocabulary (1000) were small, many tokens are still short (2-3 chars). In a full LLM like GPT-4, tokens often represent whole words or recurring phrases.

In [ ]:
tokenizer.decode(tokenizer.encode("This is some text."))

'This is some text.'

# 4. My Takeaways

- **BPE balances size vs. coverage**: It covers any text (using base characters) but compresses common text (High frequency words).
- **Training is just counting**: The "learning" part is just statistical counting of frequent pairs.
- **Encoding is recursive**: We repeatedly apply the learned merges.
- **Reversibility**: Since we just swapped pairs for IDs, we can always swap back (perfect reconstruction).

This simple implementation helped me demystify what's happening inside `tiktoken`.